# Chance accuracy

### Custom functions

In [1]:
from scipy.stats import poisson

# import spikeinterface as si
# import shutil
# from spikeinterface import comparison
# from src.nodes.postpro.cell_matching import get_SpikeInterface_matching_object


def get_p_chance_hit(fr: float, delta_time: float):
    """derive the chance agreement score of hits
    (coincidence between sorted and ground truth
    units' spike trains)

    Args:
        fr (float): in spikes/ms
        delta_time (float): SpikeInterface delta_time interval in ms

    Returns:
        _type_: _description_
    """
    k = 0  # we want the probability of k=0 coincidences
    interval_ms = 2 * delta_time  # time interval in ms
    n_sp = interval_ms * fr  # expected nb of coincidences
    return 1.0 - poisson.pmf(k=k, mu=n_sp)


def get_unit_chance_agreement_score(
    fr_gt: float, fr_s: float, rec_dur: float, p_chance_hit: float
):
    """get unit chance scorey

    The chance scorey metrics should change with the ground truth firing rate.
    It is not the case with the current calculation.
    Intuition: the more a ground truth unit spikes within the duration of recording (say 600 secs),
    the more spikes will be missed when compared a sorting unit of a fixed firing rate.
    The increasing number of misses should decrease the value of the chance score metrics,
    which is currently not the case.

    Args:
        fr_gt (float): ground truth firing rate (spikes/secs)
        fr_s (float): sorted unit firing rate (spikes/secs)
        p_chance_hit (float): chance probability of hits
        rec_dur (float): recording duration
    """
    # nb of spikes
    n_gt = fr_gt * rec_dur
    n_s = fr_s * rec_dur

    # nb of hits, false positives, misses
    # - the smallers spike train min(n_gt, n_s) determines
    # the maximum possible number of hits
    n_h = p_chance_hit * min(n_gt, n_s)
    n_fp = n_s - n_h
    n_m = n_gt - n_h
    return n_h / (n_h + n_m + n_fp)

### Calculate for an example

In [3]:
# calculate chance score for a 1 Hz sorted unit (spikes/secs)
fr_gt = 1
fr_s = 1
delta_time = 1.3  # in ms
rec_dur = 600  # recording duration

# calculate chance agreement score
# - chance probability of hits
# - chance score
p_chance_hit = get_p_chance_hit(1 / 1000, 1.3)
chance_acc = get_unit_chance_agreement_score(fr_gt, fr_s, 600, p_chance_hit)
chance_acc

0.0012999992676670962

## References

**calculation of agreement score**: https://spikeinterface.readthedocs.io/en/0.100.5/modules/comparison.html?highlight=agreement%20score#more-information-about-hungarian-or-best-match-methods